In [124]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import random
import re
from dateutil.parser import parse
from collections import defaultdict
from sklearn.linear_model import * 
from functools import reduce
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [2]:
## PLAN:
## 1) Train a model on Train Set
## 2) Validate to minimize MSE and select features
## 3) Test on remaining data to check performance

In [3]:
data = []
column_names = ['ID','country','description','designation','points','price','province','region_1','region_2','taster_name','twitter_handle','title','variety','winery']

with open('data/winemag-data-130k-v2.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if(row[0]==''):
            continue
        data.append(dict(zip(column_names,row)))

In [24]:
data[0]

{'ID': '0',
 'country': 'Italy',
 'description': "Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 'designation': 'Vulkà Bianco',
 'points': '87',
 'price': '',
 'province': 'Sicily & Sardinia',
 'region_1': 'Etna',
 'region_2': '',
 'taster_name': 'Kerin O’Keefe',
 'twitter_handle': '@kerinokeefe',
 'title': 'Nicosia 2013 Vulkà Bianco  (Etna)',
 'variety': 'White Blend',
 'winery': 'Nicosia'}

In [74]:
shuffle(data)

In [68]:
X = []
y = []
required_col_names = ['country','description','designation','price','province','region_1','region_2','taster_name','twitter_handle','title','variety','winery']

for datum in data:
    X.append( { k:v for (k,v) in datum.items() if k != 'points'})
    y.append(datum['points'])

In [89]:
X_train = X[:3*len(X)//5]
X_test = X[3*len(X)//5:4*len(X)//5]
X_val = X[4 *len(X)//5:]

y_train = y[:3*len(X)//5]
y_test = y[3*len(X)//5:4*len(X)//5]
y_val = y[4*len(X)//5]

In [90]:
y_train[0]

'85'

In [91]:
for col_name in data[0]:
    print(col_name + ': ' + str(len(list(set([datum[col_name] for datum in data])))))

ID: 129971
country: 44
description: 119955
designation: 37980
points: 21
price: 391
province: 426
region_1: 1230
region_2: 18
taster_name: 20
twitter_handle: 16
title: 118840
variety: 708
winery: 16757


In [92]:
one_hot_features = ['country', 'designation', 'province', 'region_1', 'region_2', 'variety', 'winery']
one_hot_bins = {k: set([d[k] for d in data]) for k in one_hot_features}

def featurize(data, keys):
    def feat(datum, keys):
        feature_vector = [[int(datum[k] == b) for b in one_hot_bins[k]] for k in keys if k in one_hot_features]
        return reduce(lambda a, b: a + b, feature_vector) if len(feature_vector) > 0 else []
    
    one_hot = list(map(lambda x: feat(x, keys), tqdm(data)))
    
    #if 'year' in keys:
    #    year = list(map(lambda x: find_year(x['title'])[0], data))
    #    return [[1] + [y] + hot for y, hot in zip(year, one_hot)]
    
    return one_hot

In [93]:
one_hot_bins.keys()

dict_keys(['country', 'designation', 'province', 'region_1', 'region_2', 'variety', 'winery'])

In [171]:
X = featurize(X_train[:100], ['variety', 'country'])
# X_year = featurize(X_train[:100], ['year'])

y = [points for points in y_train[:100]]


100%|██████████| 100/100 [00:00<00:00, 1426.44it/s]

In [172]:
reg = LinearRegression()
reg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [173]:
len(X[0])

752

In [163]:
def mse(predictions, labels):
    return sum([(predictions[i]-labels[i]) **2 for i in range(len(labels))])/len(labels)

In [157]:
p = reg.predict(X)


In [158]:
p.shape

(100,)

In [159]:
p[0]

85.0

In [164]:
mse(p,y)

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')